# Búsqueda de los posibles incios y finales

### Autora: Lucía Núñez Calvo

#### Fecha: 27 de Abril de 2022

Este cuaderno parte de la idea de localizar varias secuencias similares al patrón de referencia. En este caso la idea que se va a plantear es la siguiente. A partir de una secuencia de ejercicios contretos, se extraerán los inicios y fianles del ejercicio. Una vez extraídos y almacenados se procesará cada una de las posibles combinaciones entre dichos valores para obtener la secuencia más parecida posible al patrón de referencia. 

La forma de actuar será la siguiente. Imaginémonos que tenemos una secuencia de 969 de ejercicios concretos de frames y una secuencia de varios ejercicios de 2896 y decidimos destinar los 100 primeros como secuencia de inicio del ejercicio y los 100 últimos como secuencia de final. Tras aplicar la búsqueda de secuencias por los inicios y por los finales nos informa de que tenemos 3 posibles inicio y 4 posibles finales, algo parecido a lo siguiente:
- Posible inicio 1: Del frame 237 - 307
- Posible inicio 2: Del frame 214 - 294
- Posible inicio 3: Del frame 167 - 207

- Posible final 1: Del frame 1216 - 1300
- Posible final 2: Del frame 1107 - 1197
- Posible final 3: Del frame 1221 - 2310
- Posible final 4: Del frame 1250 - 1340

Una vez obtenidos estos datos el objetivo es, combinar cada uno de los posibles inicios con cada uno de los posibles finales y analizar como de parecida es la secuencia generada respecto al patrón de referencia. Es decir, para el inicio 1 se crearán cuatro secuencias:
- Secuencia 1 : Desde el frame 237 hasta el frame 1300
- Secuencia 2 : Desde el frame 237 hasta el frame 1197
- Secuencia 3 : Desde el frame 237 hasta el frame 2310
- Secuencia 4 : Desde el frame 237 hasta el frame 1340

Sobre estas secuencias se aplicarán una serie de parámetros para filtrarlas y se calcularán las distancias y costes respecto al patrón de referencia.

In [1]:
import pickle
import numpy as np

def extrat_pickle(file):
    pos=[]
    with open("/.Pickle/"+file, "rb") as f:
        while True: 
            try:
                current_id=pickle.load(f)
                pos.append(current_id)
            except EOFError:
                break
    return pos

In [2]:
ang_ej1 = extrat_pickle("angulos_ej1_MultiplesSecuencias.pickle")
ang_paciente = extrat_pickle("angulos_total_MultiplesSecuencias.pickle")

print("Número de frames del ejercicio 1: ",len(ang_ej1))
print("Número de frames del ejercicio completo: ",len(ang_paciente))

Número de frames del ejercicio 1:  969
Número de frames del ejercicio completo:  2896


In [3]:
from scipy.signal import find_peaks

from tslearn import metrics
from tslearn.generators import random_walks
from tslearn.preprocessing import TimeSeriesScalerMeanVariance


def calcule_matrix(X,Y):
    """
    Función que calcula la matriz de costes locales o matriz de distancias.
    Este cálculo se ha realizao mediante la distancia euclidiana.
    
    Parámetros de entrada 
    ---------------------
    X : Secuencia corta con un ejercicio concreto.
    Y : Secuencia larga con múltiples ejercicios.
    
    Salida
    ------
    Matriz de costes locales o de distancias.
    """
    C=np.zeros((len(X),len(Y)))
    for i in range(len(X)):
        for j in range(len(Y)):
            x=np.array(X[i])
            y=np.array(Y[j])
            
            #Cálculo de la distancia euclídea entre dos secuencias.
            C[i][j] = np.linalg.norm(np.array(X[i])-np.array(Y[j])) #para secuencias de varias dimensiones

    return C

def calcule_matrixD(C,X,Y):
    """
    Función que calcula la matriz de costes acumulados.
    Este cálculo se realizará de la siguiente forma:
        1) La primera fila de D está inicializada como D(1,m):=C(1,m) para m € [1:M] siendo M=len(Y).
        2) La primera columna de D está inicializada como D(n,1):=∑k=1nC(k,1) para n € [1:N] siendo N=len(X).
        3) El resto de los valores se definen recursivamente a partir de la matriz de costes locales.
    
    Parámetros de entrada 
    ---------------------
    C : Matriz de costes locales o de distancias.
    X : Secuencia corta con un ejercicio concreto.
    Y : Secuencia larga con múltiples ejercicios.
    
    Salida
    ------
    Matriz de costes acumulados.
    """
    D=np.zeros((len(X),len(Y)))
    D[:,0]=np.cumsum(C[:,0]) #inicialización de la primera columna
    D[0,:]=C[0,:] #Inicialización de la primera fila 

    for i in range(1,len(X)):
        for j in range(1,len(Y)):
            D[i][j] = C[i][j]+min(D[i-1][j],D[i][j-1],D[i-1][j-1])
            
    return D

def calcule_path(C,sz):
    """
    Función que calcula las posibles rutas de deformación.
    Estas rutas serán calculadas mediante la función scipy.signal.find_peaks que localiza picos dentro
    de una señal. Con el argumento "distance" se especifica distancia horizontal mínima entre picos, cuanto 
    más restrictivo sea este parámetro, menor será el número de caminos localizados.
    
    Parámetros de entrada 
    ---------------------
    C : Matriz de costes locales o de distancias.
    sz : Parámetro que se utiliza para hacer más o menos restrictiva la búsqueda.
    
    Salida
    ------
    Posibles rutas de deformación.
    """
    #Calculo del coste de la función
    cost_func = C[-1, :]

    #Identificación de los posibles caminos
    potential_matches = find_peaks(-cost_func, distance=sz * 2.75, height=-50)[0]

    #Calculo de las rutas óptimas a partir de cada uno de los mínimos identificados
    paths = [metrics.subsequence_path(C, match) for match in potential_matches]
    
    return paths

Extraemos los posibles inicios y finales de la secuencia. Para ello se ha cogido el 10% de los frames como longitud de inicio y final de la secuencia.

In [8]:
longitud=int(0.1*len(ang_ej1))

inicio_ej1=ang_ej1[:longitud]
fin_ej1=ang_ej1[-longitud:]

In [19]:
import time
inicio = time.time()

C=calcule_matrix(inicio_ej1, ang_paciente)
#print("\nMatriz de distancias o matriz de costes locales con sqrt(x[i] - y[j])^2:\n ",C) 

valores_sz=[3,7,10,14,20,30,40,50,60,70]

for sz in valores_sz:
    paths=calcule_path(C,sz)
    print("Número de posibles inicios para sz = "+str(sz)+": ",len(paths))
    
fin = time.time()
print("\nTiempo trascurrido en localizar los posibles caminos óptimos: ",fin-inicio, "segundos")

Número de posibles inicios para sz = 3:  98
Número de posibles inicios para sz = 7:  66
Número de posibles inicios para sz = 10:  56
Número de posibles inicios para sz = 14:  42
Número de posibles inicios para sz = 20:  30
Número de posibles inicios para sz = 30:  22
Número de posibles inicios para sz = 40:  19
Número de posibles inicios para sz = 50:  17
Número de posibles inicios para sz = 60:  11
Número de posibles inicios para sz = 70:  10

Tiempo trascurrido en localizar los posibles caminos óptimos:  3.9211032390594482 segundos


Una vez calculadas las rutas de los supuestos inicios, se almacena únicamente el valor de comienzo

In [20]:
supuestos_inicios=[]

for p in paths:
    p=np.array(p)
    a_ast = p[0, 1]
    supuestos_inicios.append(a_ast)
    
print("Supuestos inicios: ",supuestos_inicios)

Supuestos inicios:  [104, 375, 557, 838, 1153, 1469, 1654, 1950, 2356, 2668]


In [21]:
import time
inicio = time.time()

C=calcule_matrix(fin_ej1, ang_paciente)
#print("\nMatriz de distancias o matriz de costes locales con sqrt(x[i] - y[j])^2:\n ",C) 

valores_sz=[3,7,10,14,20,30,40,50,60,70]

for sz in valores_sz:
    paths=calcule_path(C,sz)
    print("Número de posibles finales para sz = "+str(sz)+": ",len(paths))

fin = time.time()
print("\nTiempo trascurrido en localizar los posibles caminos óptimos: ",fin-inicio, "segundos")

Número de posibles finales para sz = 3:  102
Número de posibles finales para sz = 7:  72
Número de posibles finales para sz = 10:  58
Número de posibles finales para sz = 14:  44
Número de posibles finales para sz = 20:  33
Número de posibles finales para sz = 30:  22
Número de posibles finales para sz = 40:  19
Número de posibles finales para sz = 50:  14
Número de posibles finales para sz = 60:  13
Número de posibles finales para sz = 70:  11

Tiempo trascurrido en localizar los posibles caminos óptimos:  3.8453996181488037 segundos


Una vez calculadas las rutas de los supuestos finales, se almacena únicamente el valor de final

In [22]:
supuestos_finales=[]

for p in paths:
    p=np.array(p)
    b_ast = p[-1, 1]
    supuestos_finales.append(b_ast)
    
print("Supuestos finales: ",supuestos_finales)

Supuestos finales:  [122, 324, 633, 898, 1110, 1383, 1767, 2009, 2204, 2550, 2762]


Una vez extraídos los posibles inicios y finales, el siguiente paso será localizar cada una de las posibles rutas entre los distintos inicios y finales. Un aspecto muy importante a tener en cuenta es el tamaño del camino encontrado. Si por ejemplo el algoritmo detecta que un posible inicio es el frame 200 y un posible final es el frame 400 en un ejercicio que consta de 800 frames, no tendría mucho sentido este resultado. Es por esta razón que lo primero que se hará es comprobar si la secuencia localizada tiene un tamaño parecido a la secuencia esperada. 

In [26]:
from tslearn.metrics import dtw_subsequence_path
from dtaidistance import dtw_ndim
from pydtw import dtw2d
from tslearn.metrics import dtw_path_from_metric
from tslearn.metrics import dtw, dtw_path
from tslearn.metrics import soft_dtw

def classification(short_seq, long_seq, supuestos_inicios, supuestos_finales):
    """
    Función que calcula las distintas distancias y costes entre el patrón de referencia y la secuencia encontrada.
    Es decir, esta función funciona de la siguiente manera:
        1) Recibe el patrón de referencia.
        2) Recibe la secuencia de ejercicios completa.
        3) Recibe los posibles inicios y finales.
        4) Crea y recorre cada uno de los posibles caminos.
            4.1) Por cada camino saca el punto de inicio y final de la secuencia concreta dentro de la secuencia 
                de mayor tamaño.
            4.2) Calcula las distancias y costes entre la posible secuencia y la secuencia real.
    
    La tasa de error aplicada para la localización de una secuencia similar, se calcula como un porcentaje
    de la secuencia real, de esta manera se admitirán secuencias que sean un poco más pequeñas o un poco 
    más grandes que la secuencia real.
    
    Parámetros de entrada 
    ---------------------
    short_seq : Secuencia corta con un ejercicio concreto.
    long_seq : Secuencia larga con múltiples ejercicios.
    paths : Posibles caminos encontrados.
    """
    #Calculo de la cota de error a partir de la secuencia de referencia
    lon_ej1=len(short_seq)
    error=lon_ej1*0.2 
    #Extracción de cada uno de los posibles caminos
    for a in supuestos_inicios:
        for b in supuestos_finales:
            #Filtración según tamaño
            diferencia=b-a
            
            if a < b and diferencia < lon_ej1+error and diferencia > lon_ej1-error:
                #Cálculo de las distintas distancias y costes
                print("\nInicio: ",a)
                print("Final: ",b)

                serie1=np.array(short_seq)
                serie2=np.array(long_seq[a:b+1])

                path, dist = dtw_subsequence_path(serie1, serie2)
                print("Distancia 1: ",dist)

                distance = dtw_ndim.distance(serie1,serie2)
                print("Distancia 2: ",distance)

                euclidean_distance = dtw_ndim.ub_euclidean(serie1,serie2)
                print("Euclidean_distance: ",euclidean_distance)

                cost_matrix, cost, alignmend_a, alignmend_b = dtw2d(serie1,serie2)
                print('Coste 1: ',cost)

                optimal_path, cost = dtw_path_from_metric(serie1, serie2)
                print("Coste 2: ",cost)

                cost = dtw(serie1, serie2, global_constraint="sakoe_chiba", sakoe_chiba_radius=0.5)
                print("Coste 3: ",cost)

                cost = dtw(serie1, serie2, global_constraint="itakura", itakura_max_slope=2.)
                print("Coste 4: ",cost)

                soft_dtw_score = soft_dtw(serie1, serie2, gamma=1.5)
                print("Soft: ",soft_dtw_score)

In [27]:
classification(ang_ej1, ang_paciente, supuestos_inicios, supuestos_finales)


Inicio:  104
Final:  898
Distancia 1:  4536.942121380077
Distancia 2:  4829.38088715872
Euclidean_distance:  6054.671197787689
Coste 1:  inf
Coste 2:  136508.25414841843
Coste 3:  4850.324890896184
Coste 4:  4835.228976357514
Soft:  23322919.753253903

Inicio:  104
Final:  1110
Distancia 1:  4494.443691602438
Distancia 2:  5057.66270083867
Euclidean_distance:  6256.584022532164
Coste 1:  inf
Coste 2:  150683.19558501325
Coste 3:  5109.529602186919
Coste 4:  5064.500829739841
Soft:  25579951.9954547

Inicio:  375
Final:  1383
Distancia 1:  4449.695970321573
Distancia 2:  5009.229630663472
Euclidean_distance:  6209.222665606398
Coste 1:  86576311.83170791
Coste 2:  148212.14072588133
Coste 3:  5024.371191649926
Coste 4:  5014.854576124558
Soft:  25092381.49267285

Inicio:  557
Final:  1383
Distancia 1:  4449.695970321573
Distancia 2:  4763.660707771917
Euclidean_distance:  6138.050930944237
Coste 1:  73403024.4101774
Coste 2:  135262.70832350448
Coste 3:  4763.660707771917
Coste 4:  477

<div class="alert alert-block alert-warning">
    <b>Conclusión: </b> Como se puede observar las combinaciones de incio y final son muy diferentes unas de otras pero sus distancias y costes no lo son tanto, hay en ocasiones que incluso son iguales. 

Se sabe que el ejercicio 1 abarca los 960 frames iniciales dentro de la secuencia que comprende todos los ejercicios. Como se puede observar en los resultados anteriores, a grandes rasgos las tres primeras secuencias podrían valer pero el problema está en que los valores de distancia y coste de esas secuencias, con prácticamente iguales a los de las secuencias incorrectas. 
    
Por lo tanto, esta medida no serviría para realizar un una buena localización de la secuencia esperada.
    
</div>